In [ ]:
#pip install asf_search


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import requests
from tqdm import tqdm

# ---- USER CONFIG ----
INPUT_CSV = "/content/drive/MyDrive/NASA_SPACE_APP/Chesapeake_SAR_Envi_Multi_Date_548_dates.csv"
OUTPUT_CSV = "/content/drive/MyDrive/NASA_SPACE_APP/SAR_with_NASA_POWER_full.csv"

POWER_PARAMS = [
    "T2M",                # "air_temp_2m_C"
    "T2M_MAX",            # "max_air_temp_2m_C"
    "T2M_MIN",            # "min_air_temp_2m_C"
    "WS2M",               # "wind_speed_2m_m/s"
    "PRECTOTCORR",        # "precipitation_mm"
    "ALLSKY_SFC_SW_DWN"   # "solar_radiation_MJ/m2/day"
]
PARAM_STRING = ",".join(POWER_PARAMS)
COMMUNITY = "ag"

POWER_COL_RENAME = {
    "T2M": "nasa_power_air_temp_2m",
    "T2M_MAX": "nasa_power_max_air_temp_2m",
    "T2M_MIN": "nasa_power_min_air_temp_2m",
    "WS2M": "nasa_power_wind_speed_2m_m/s",
    "PRECTOTCORR": "nasa_power_precipitation_mm",
    "ALLSKY_SFC_SW_DWN": "nasa_power_solar_radiation_MJm2day"
}

# ---- LOAD ALL SAR POINTS ----
sar_df = pd.read_csv(INPUT_CSV)
sar_df['date_fmt'] = pd.to_datetime(sar_df['date']).dt.strftime('%Y%m%d')
sar_df.columns = [c.strip().lower() for c in sar_df.columns]

def fetch_power_json(lat, lon, date_str):
    url = (
        f"https://power.larc.nasa.gov/api/temporal/daily/point?"
        f"parameters={PARAM_STRING}&community={COMMUNITY}"
        f"&longitude={lon}&latitude={lat}"
        f"&start={date_str}&end={date_str}&format=JSON"
    )
    try:
        response = requests.get(url, timeout=20).json()
        param_block = response['properties']['parameter']
        result = {POWER_COL_RENAME[p]: param_block.get(p, {}).get(date_str, None) for p in POWER_PARAMS}
        return result
    except Exception as e:
        print(f"Error for {lat},{lon},{date_str}: {e}")
        return {POWER_COL_RENAME[p]: None for p in POWER_PARAMS}

records = []

print(f"Fetching NASA POWER data for ALL {len(sar_df)} SAR rows...")

for idx, row in tqdm(sar_df.iterrows(), total=len(sar_df)):
    power_data = fetch_power_json(row['latitude'], row['longitude'], row['date_fmt'])
    out_row = {**row.to_dict(), **power_data}
    records.append(out_row)

out_df = pd.DataFrame(records)
out_df.drop(columns=['date_fmt'], inplace=True, errors='ignore')
out_df.to_csv(OUTPUT_CSV, index=False)
print(f"File saved as: {OUTPUT_CSV}")


Fetching NASA POWER data for ALL 9381 SAR rows...


 96%|█████████▌| 8995/9381 [4:16:26<11:20,  1.76s/it]

In [ ]:
#records

In [ ]:
OUTPUT_CSV_2="/content/drive/MyDrive/NASA_SPACE_APP/SAR_NASA_POWER_2.csv"
out_df.to_csv(OUTPUT_CSV_2, index=False)